In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
root = '../'
df = pd.read_csv(root+'order_brush_order.csv')
df_c_order = df.groupby('shopid').count()
# shop with more than 3 buyers overall
df[~df['shopid'].isin(df_c_order[df_c_order['orderid']<3].index)].sort_values(by='shopid')

,orderid,shopid,userid,event_time
200439,31412115824794,10061,130633421,2019-12-30 21:35:15
145426,31431527100615,10061,62464559,2019-12-31 02:58:48
188513,31195675919209,10061,168750452,2019-12-28 09:27:55
174620,31205132327893,10061,194819216,2019-12-28 12:05:32
109999,31410842885758,10084,136264815,2019-12-30 21:14:03
...,...,...,...,...
110351,31328220560965,213900783,124465026,2019-12-29 22:17:00
83893,31506349293329,214432425,52594422,2019-12-31 23:45:49
133194,31505523761333,214432425,1134243,2019-12-31 23:32:03
4590,31506394434087,214432425,52594422,2019-12-31 23:46:35


In [4]:
df.sort_values(by='event_time')[['shopid','userid','event_time']].values.tolist()

[[6042309, 97707522, '2019-12-27 00:00:00'],
 [104804492, 97707522, '2019-12-27 00:00:00'],
 [8715449, 97707522, '2019-12-27 00:00:00'],
 [190969466, 170182475, '2019-12-27 00:00:02'],
 [2859407, 12532131, '2019-12-27 00:00:05'],
 [94479614, 12532131, '2019-12-27 00:00:05'],
 [65551316, 12532131, '2019-12-27 00:00:05'],
 [147941492, 195903090, '2019-12-27 00:00:08'],
 [54615708, 128755891, '2019-12-27 00:00:08'],
 [127236302, 128755891, '2019-12-27 00:00:08'],
 [137739819, 128755891, '2019-12-27 00:00:08'],
 [26218382, 128755891, '2019-12-27 00:00:08'],
 [18784836, 128755891, '2019-12-27 00:00:08'],
 [57934775, 91112850, '2019-12-27 00:00:11'],
 [31733346, 91112850, '2019-12-27 00:00:11'],
 [9374147, 214198595, '2019-12-27 00:00:11'],
 [147941492, 210365192, '2019-12-27 00:00:12'],
 [29975426, 146126513, '2019-12-27 00:00:16'],
 [316121, 209817190, '2019-12-27 00:00:16'],
 [141528773, 209817190, '2019-12-27 00:00:16'],
 [112904482, 146126513, '2019-12-27 00:00:16'],
 [68862371, 450199,

In [5]:
from datetime import *

checklist, contribute_count, record = dict(), dict(), dict()
for shop, user, time in df.sort_values(by='event_time')[['shopid','userid','event_time']].values.tolist() :
    if shop not in checklist :
        checklist[shop] = []
        contribute_count[shop] = dict()
    record[shop] = '0'
    time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    checklist[shop].append((user,time))
    if user not in contribute_count[shop] :
        contribute_count[shop][user] = 0
    contribute_count[shop][user] += 1

def printlist(shop):
    for order in checklist[shop] :
        print(order)

def find_max_contribute(shop) :
    user_list = []
    for user in sorted(contribute_count[shop].keys()) :
        if contribute_count[shop][user] == max(contribute_count[shop].values()) :
            user_list.append(str(user))
    return '&'.join(user_list)
print('Success')

Success


In [6]:
dead_list = []
for shop in sorted(checklist.keys()) :
    if len(checklist[shop]) > 2 :
        start, end = 0, 0
        unique_customers = [checklist[shop][0][0]]
        while True :
            #print(unique_customers)
            end += 1
            if end == len(checklist[shop]) : break
            else :
                #print(unique_customers)
                if checklist[shop][end][0] not in unique_customers :
                    unique_customers.append(checklist[shop][end][0])
                for i in range(start,end):
                    time_diff = checklist[shop][end][1] - checklist[shop][i][1]
                    if time_diff > timedelta(hours=1) :
                        if checklist[shop][i+1][0] != checklist[shop][i][0] and len(unique_customers) > 0:
                            unique_customers.pop(0)
                    else :
                        start = i
                        break 
                if not changed : start = end - 1
                if len(unique_customers) > 0 and (end - start) // len(unique_customers) >= 3 :
                    record[shop] = find_max_contribute(shop)
                    #print("Shop :",shop,'Substituted')
                    dead_list.append(shop)
                    break
            
    #print('Shop',shop,"Done.")
print('Success')

NameError: name 'changed' is not defined

In [ ]:
print(len(dead_list))
for id in dead_list :
     print(id, record[id])

In [ ]:
X = []
for shopid in (record.keys()) :
    X.append([shopid,record[shopid]])

result = pd.DataFrame(X, columns=['shopid','userid'])

result.to_csv('submission.csv',header=True,index=False)